# Create a New Folder #

In [20]:
import os
os.getcwd()

'/usr/local/NNShared/hali'

In [44]:
os.chdir("/usr/local/NNShared/")
os.listdir()

newpath = "/usr/local/NNShared/hali/exceptions" 
if not os.path.exists(newpath):
    os.makedirs(newpath)

path = "/usr/local/NNShared/hali/ebooks"
if not os.path.exists(path):
    os.makedirs(path)

# Now Moving the Txt File #

In [45]:
os.chdir("/usr/local/NNShared/hali")

In [10]:
import os

dir = "/usr/local/NNShared/hali/exceptions"
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

Check for the results:

In [118]:
import os
os.chdir("/usr/local/NNShared/hali")
print(len(os.listdir("/usr/local/NNShared/hali")))
print(len(os.listdir("/usr/local/NNShared/hali/ebooks")))
print(len(os.listdir("/usr/local/NNShared/hali/exceptions")))
print(len(os.listdir("/usr/local/NNShared/hali/data")))

3
36100
43
1


# Find the language tag for each book #

In [120]:
import re
import os
import shutil

languages = []

for file in os.listdir("/usr/local/NNShared/hali/ebooks"):
    thisfile = open("/usr/local/NNShared/hali/ebooks/"+file, "r", encoding = "ISO-8859-1")
    
    flag = 0
    
    for x in thisfile:
        y = re.search('Language:', x)
        if y!= None:
            languages.append(y.string.rstrip().split(": ")[1])
            flag = 1
            break
    if flag == 0:
        shutil.copy("/usr/local/NNShared/hali/ebooks/"+file, "/usr/local/NNShared/hali/exceptions")
        os.remove("/usr/local/NNShared/hali/ebooks/"+file)

print(len(os.listdir("/usr/local/NNShared/hali/ebooks")))
print(len(os.listdir("/usr/local/NNShared/hali/exceptions")))

36099
44


In [121]:
import os 
os.chdir("/usr/local/NNShared/hali/ebooks")

books = []

for file in os.listdir("/usr/local/NNShared/hali/ebooks"):
    if file == '20060205-871-8.txt':
        books.append('20060205-871')
    elif file == '20100823-8387-8.txt':
        books.append('20100823-8387')
    elif file == '20080731-834-8.txt':
        books.append('20080731-834')
    else:
        books.append(file.split("-8")[0])
        
    
print(len(books))

36099


# Save them as a json file #

In [122]:
bookandlang = list(zip(books, languages))

In [123]:
import json

newpath = "/usr/local/NNShared/hali/data" 
if not os.path.exists(newpath):
    os.makedirs(newpath)

with open("/usr/local/NNShared/hali/data/data.json", 'w') as f:
    json.dump(bookandlang, f)

In [1]:
#read the json file#

import json

json_file_path = "/usr/local/NNShared/hali/data/data.json"

with open(json_file_path, 'r') as j:
     contents = json.loads(j.read())

print(len(contents))

36099


# Remove the header and footer of each book #

In [2]:
import os
import re

newpath = "/usr/local/NNShared/hali/newebooks" 
if not os.path.exists(newpath):
    os.makedirs(newpath)

for pairs in contents:
    f = open("/usr/local/NNShared/hali/ebooks/"+pairs[0]+"-8.txt", "r", encoding = "ISO-8859-1")

    flag = 0

    sample = []

    for myline in f:
        y = re.search("\\*\\*\\* START OF", myline)
        z = re.search("\\*\\*\\* END OF",myline)
        a = re.search("\\*END THE SMALL PRINT", myline)
        b = re.search("End of the Project", myline)
        p = re.search("\\*\\*\\*START OF", myline)
        q = re.search("\\*\\*\\*END OF", myline)

        if z:
            flag = 0
        elif b:
            flag = 0
        elif q:
            flag = 0
            
        if flag == 1:
            if myline != '\n':
                sample.append(myline.rstrip())
        if y:
            flag = 1
        elif a:
            flag = 1
        elif p:
            flag = 1
    
    with open("/usr/local/NNShared/hali/newebooks/"+pairs[0]+".txt", 'w') as file:
        file.write(' '.join(sample))

print(len(os.listdir("/usr/local/NNShared/hali/newebooks")))

36099


# Remove those books which does not have standard header and footer format #

In [7]:
import os

for files in os.listdir("/usr/local/NNShared/hali/newebooks"):
    books = open("/usr/local/NNShared/hali/newebooks/"+files,"r")
    #open each new book#
        
    length = len(books.read())
        
    if length < 150:
        os.remove("/usr/local/NNShared/hali/newebooks/"+files)

print(len(os.listdir("/usr/local/NNShared/hali/newebooks")))

36082


# Match the left books with their language tag and save as a dictionary #

In [17]:
import json
import os

json_file_path = "/usr/local/NNShared/hali/data/data.json"

with open(json_file_path, 'r') as j:
        contents = json.loads(j.read())

bookswithlang = {}
        
for newbooks in os.listdir("/usr/local/NNShared/hali/newebooks"):
    for books in contents:
        if newbooks.split(".t")[0] == books[0]:
            bookswithlang[newbooks.split(".t")[0]] = books[1]

print(len(bookswithlang))

36082


In [16]:
#check if the number of books matches #

a = []

for newbooks in os.listdir("/usr/local/NNShared/hali/newebooks"):
    flag = 0
    for books in contents:
        if newbooks.split(".t")[0] != books[0]:
            flag = flag + 1
    if flag == 36099:
        print(newbooks)

In [18]:
#check the json files #

import json

with open("/usr/local/NNShared/hali/data/cleanBL.json", 'w') as f:
    json.dump(bookswithlang, f)

print(os.listdir("/usr/local/NNShared/hali/data"))

['data.json', 'cleanBL.json']


In [16]:
#read our dictionary#
import json
f = open("/usr/local/NNShared/hali/data/cleanBL.json", 'r')

dic = json.load(f)

f.close()

# Vectorize each character in each book #

In [10]:
import os
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
import json

texts = []
labels = []
dictionary = json.load(open("/usr/local/NNShared/hali/data/cleanBL.json", 'r'))

for books, languages in dictionary.items():
    labels.append(languages)
    with open("/usr/local/NNShared/hali/newebooks/"+books+".txt","r") as f:
        text = f.read()[50:200]
        texts.append(text)
        f.close()

In [12]:
tokenizer = Tokenizer(char_level=True)

tokenizer.fit_on_texts(texts)

sequences = tokenizer.texts_to_sequences(texts)

one_hot_results = tokenizer.texts_to_matrix(texts, mode='binary')

dic = tokenizer.word_index

def string_vectorizer(strng, dic):
    vector = [[0 if char != letter.lower() else 1 for char in dic] 
                  for letter in strng]
    return vector
 
lst = []

for i in range(len(texts)):
    
    lst.append(np.reshape(string_vectorizer(texts[i], dic),(150,len(dic)))) 
    
print(len(np.array(lst)))


{' ': 1, 'e': 2, 't': 3, 'a': 4, 'i': 5, 'r': 6, 'n': 7, 'o': 8, 's': 9, 'd': 10, 'l': 11, 'p': 12, 'h': 13, 'm': 14, 'g': 15, 'u': 16, 'w': 17, 'b': 18, '.': 19, 'c': 20, 'f': 21, 'y': 22, '/': 23, 'v': 24, '-': 25, ':': 26, 'k': 27, ',': 28, 'j': 29, '(': 30, '1': 31, ')': 32, '_': 33, 'ä': 34, "'": 35, '"': 36, '9': 37, '8': 38, '[': 39, '*': 40, 'x': 41, '0': 42, ']': 43, '2': 44, 'z': 45, 'q': 46, '=': 47, 'ö': 48, '3': 49, '7': 50, '5': 51, 'é': 52, '4': 53, '6': 54, 'è': 55, '&': 56, '+': 57, ';': 58, '\t': 59, 'ü': 60, '?': 61, '!': 62, '#': 63, '|': 64, 'ó': 65, 'ã': 66, 'ç': 67, 'á': 68, 'å': 69, '@': 70, 'í': 71, 'à': 72, '·': 73, 'æ': 74, 'ñ': 75, '»': 76, 'º': 77, '{': 78, 'ê': 79, 'â': 80, 'ë': 81, '«': 82, '}': 83, '<': 84, '¡': 85, 'ð': 86, '°': 87, '¤': 88, '~': 89, 'ò': 90, 'ß': 91, '©': 92, '>': 93, 'î': 94, 'ô': 95, 'õ': 96, 'ï': 97, '\xa0': 98, '^': 99, '¨': 100, 'ø': 101, 'û': 102, 'ú': 103, '\x92': 104, '¥': 105, '$': 106, '\x80': 107, 'ì': 108, 'ª': 109, '§': 11

In [13]:
print(np.array(lst).shape)

(36082, 150, 160)


In [11]:
print(np.array(lst))

NameError: name 'lst' is not defined

In [11]:
#save the data 
with open("/usr/local/NNShared/hali/data/input_X.npy", "wb") as f:
    np.save(f, np.array(lst))

# Vectorize our labels #

In [14]:
tokenizer1 = Tokenizer(num_words = 1000)

tokenizer1.fit_on_texts(labels)

sequences1 = tokenizer1.texts_to_sequences(labels)

one_hot_results1 = tokenizer1.texts_to_matrix(labels, mode='binary')

dic1 = tokenizer1.word_index

ans = np.array(string_vectorizer(labels, dic1))

{'english': 1, 'french': 2, 'finnish': 3, 'german': 4, 'dutch': 5, 'portuguese': 6, 'italian': 7, 'spanish': 8, 'swedish': 9, 'danish': 10, 'latin': 11, 'tagalog': 12, 'and': 13, 'catalan': 14, 'hungarian': 15, 'norwegian': 16, 'esperanto': 17, 'en': 18, 'greek': 19, 'afrikaans': 20, 'with': 21, 'icelandic': 22, 'polish': 23, 'friulian': 24, 'chinese': 25, 'friulan': 26, 'nu': 27, 'cebuano': 28, 'frisian': 29, 'gaelic': 30, 'nahuatl': 31, 'russian': 32, 'serbian': 33, 'flemish': 34, 'iloko': 35, 'czech': 36, 'japanese': 37, 'comments': 38, 'character': 39, 'set': 40, 'encoding': 41, 'estonian': 42, 'multiple': 43, 'inuktitut': 44, 'bagobo': 45, 'kashubian': 46, 'arapaho': 47, 'welsh': 48, 'galician': 49, 'breton': 50, 'careful': 51, 'bulgarian': 52, 'venetian': 53, 'neapolitan': 54, 'gascon': 55, 'ancient': 56, 'portugese': 57, 'old': 58, 'middle': 59, '1100': 60, '1500': 61, 'nederlands': 62, 'ilocano': 63, 'chinook': 64, 'quiche': 65, 'onondaga': 66, 'khasi': 67, 'language': 68, 'spo

In [15]:
#save the data 
with open("/usr/local/NNShared/hali/data/input_Y.npy", "wb") as f:
    np.save(f, ans)